## Neo4j Code to load data, nodes, properties and relationships

This file is an earlier version of the neo4j file that included additional exploritory steps, such as using neo4j's similarity scores.

In [1]:
## 1- Install neo4j Desktop
## 2- Create local database with authentication (user name /password)
## 3- Export csv from relational database & xml file (not needed if already csb there)
## 4- Copy Data files to neo4j data directory
##/Users/prakhar/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-<your-db-id>/installation-4.3.7/import/
## my local neo4j directory 
###ls "/Users/prakhar/Library/A7690pplication Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-c1b4f9c3-baa6-45d6-a13c-31ec4ba4c393/import"
#5 cypher queries

In [2]:
## install 
#pip install py2neo
#!pip install tensorflow_hub
#!pip install tensorflow

In [3]:
import pandas as pd
from pandas import json_normalize
import ast
from ast import literal_eval
from py2neo import Graph
import spacy

In [4]:
# Connect to your Neo4j database
graph = Graph("bolt://localhost:7690", auth=("neo4j", "Welcome19#")) #PS

In [5]:
# path to your SBIR Excel file
filename_sbir = './preprocessed_files/sbir_1k_sample.csv'

# Read the Excel file into a pandas DataFrame
df_sbir = pd.read_csv(filename_sbir)
#df_sbir.rename(columns={'Agency Tracking Number': 'id_Agency Tracking Number'}, inplace=True)
df_sbir['key'] = df_sbir.index
df_sbir.head(5)

,id,Company,Award Title,Agency,Branch,Phase,Program,Agency Tracking Number,Contract,Proposal Award Date,...,Contact Email,PI Name,PI Title,PI Phone,PI Email,RI Name,RI POC Name,RI POC Phone,abstract_entities,key
0,16068,"EXPEDITION TECHNOLOGY, INC.",Fast Recovery Of Signal Estimates using Neural...,Department of Defense,Navy,Phase I,SBIR,N192-128-0403,N68335-19-C-0742,10/21/2019,...,marc.harlacher@exptechinc.com,Mike Tinston,Chief Scientist,(571) 246-8479,mike.tinston@exptechinc.com,NaN,NaN,NaN,"['condition', 'signal density', 'signal mingle...",0
1,187914,"Apa Optics, Inc.",EXTRAVEHICULAR MOBILITY UNIT HELMET MOUNTED DI...,National Aeronautics and Space Administration,NaN,Phase II,SBIR,6818,NaN,NaN,...,NaN,David E Stoltzmann,NaN,() -,NaN,NaN,NaN,NaN,"['OPTICAL MECHANICAL DESIGNS', 'opto-mechanica...",1
2,97687,"RADIATION MONITORING DEVICES, INC.","An Efficient, Solid State Detector for Nuclear...",Department of Energy,NaN,Phase II,SBIR,80498S06-I,DE-FG02-06ER84430,NaN,...,GEntine@rmdinc.com,Michael Squillante,Dr,(617) 668-6808,MSquillante@rmdinc.com,NaN,NaN,NaN,"['signal-to-noise ratio', 'Computed tomography...",2
3,38328,"QUASAR FEDERAL SYSTEMS, INC.",Miniature Oriented Tri-Axial Fluxgate Magnetom...,Department of Defense,Navy,Phase I,SBIR,N172-116-0564,N68335-17-C-0621,08/30/2017,...,twrightson@quasarfs.com,Yongming Zhang,CEO,(858) 412-1737,yongming@quasarfs.com,NaN,NaN,NaN,"['long range sigint', 'magnetic signature', 'e...",3
4,82571,"INVOTEK, INC.",Prosody enhanced TTS for Dysarthric Speakers,Department of Health and Human Services,National Institutes of Health,Phase I,SBIR,DC010085,1R43DC010085-01,NaN,...,tjakobs@invotek.org,THOMAS JAKOBS,NaN,(479) 632-4166,TJAKOBS@INVOTEK.ORG,NaN,NaN,NaN,"['feasibility project', 'control prosody compu...",4


In [6]:
selected_columns1 = ['key',
                     'Agency Tracking Number', 'Company', 'Branch', 'Award Title', 'Agency', 'Company Website', 'Contract', 'Proposal Award Date', 'Contract End Date', 'Solicitation Number', 'Solicitation Year',
                     'Solicitation Close Date', 'Proposal Receipt Date', 'Date of Notification', 'Topic Code', 'Award Year', 'Award Amount',
                     'PI Name', 'PI Title', 'PI Phone', 'PI Email', 'RI Name', 'RI POC Name', 'RI POC Phone','Address1','Address2', 'City', 'State', 'Zip',
                     'HUBZone Owned', 'Socially and Economically Disadvantaged','Women Owned'
                    ]

selected_columns2 = ['key', 'abstract_entities']
# Remove leading/trailing whitespaces
selected_columns1 = [col.strip() for col in selected_columns1]
selected_columns2 = [col.strip() for col in selected_columns2]

df_sbirs = df_sbir[selected_columns1]

df_sbirent = df_sbir[selected_columns2]


df_sbirs.to_csv('./preprocessed_files/neo4j_sbir_main.csv', index=False)

In [7]:
df_source_sbir = pd.DataFrame(columns=['key','abstract_entities'])
for i in range(0, len(df_sbirent)):
    temp_list = df_sbirent['abstract_entities'].iloc[i].replace('\'','').replace('[','').replace(']','').split(',')

    for x in range(0, len(temp_list)):
        #print(temp_list[x])
        df_source_sbir.loc[len(df_source_sbir.index)] = [df_sbirent['key'].iloc[i], temp_list[x].strip()]
        #df_sbirent.loc[len(df_entities_sbir.index)] = [df_sbirent['key'].iloc[i], temp_list[x]]
df_exploded_sbir=df_source_sbir
print(df_exploded_sbir)

       key                                 abstract_entities
0        0                                         condition
1        0                                    signal density
2        0                                     signal mingle
3        0             signal intermodulation product desire
4        0  Bandwidth ( ibw ) receiver wide bit-depth reduce
...    ...                                               ...
20827  890                         expert design engineering
20828  890                                   sample transfer
20829  890                                        efficiency
20830  890                                       integration
20831  890                                 mechanical design

[20832 rows x 2 columns]


In [8]:
# Read in the patent data
filename_patents = './preprocessed_files/patents.json'
df_patent = pd.read_json(filename_patents)
df_patent['key'] = df_patent['doc-number']  # PS Change 12/08 12 PM
#df_patent.head(5)

# Display the DataFrame
display(df_patent)

,country,doc-number,date,application-reference,title,assignee,inventors,abstract,claims,abstract_entities,claim_entities,key
0,US,20230227158,20230720,"{'country': 'US', 'doc-number': 18123405, 'dat...",UNMANNED AERIAL SYSTEM AND METHOD FOR CONTACT ...,"{'orgname': 'Beirobotics LLC', 'city': 'Richmo...","[{'last-name': 'Beiro', 'first-name': 'Michael...",\nA system for performing work on electrical p...,a power line tool adapted to perch on an energ...,"[splice electrical power line, power line tool...","[power line tool, method, couple unmanned, sla...",20230227158
1,US,20230229051,20230720,"{'country': 'US', 'doc-number': 17577538, 'dat...",METHOD AND DEVICE FOR CONTROLLING STATES OF DY...,"{'orgname': 'FURCIFER INC.', 'city': 'FREMONT'...","[{'last-name': 'WANG', 'first-name': 'JIAN', '...",\nThe disclosure relates generally to a method...,selecting a desired optical state of the elect...,"[method, optical state base, switch, power, di...","[controller, optical state base, switch, signa...",20230229051
2,US,20230229087,20230720,"{'country': 'US', 'doc-number': 18098167, 'dat...",UV-CURABLE QUANTUM DOT FORMULATIONS,"{'orgname': 'Nanosys, Inc.', 'city': 'Milpitas...","[{'last-name': 'IPPEN', 'first-name': 'Christi...",\nProvided are patterned films comprising nano...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n,"[electroluminescent device, method pattern, uv...",[],20230229087
3,US,20230230714,20230720,"{'country': 'US', 'doc-number': 18010358, 'dat...",CONTROL DRUM CONTROLLER FOR NUCLEAR REACTOR SY...,"{'orgname': 'Ultra Safe Nuclear Corporation', ...","[{'last-name': 'Chaleff', 'first-name': 'Ethan...",\nA nuclear reactor system includes a nuclear ...,a pressure vessel;\na nuclear reactor core dis...,"[moderator element, nuclear reactor core dispo...","[internal control drum, longitudinally pressur...",20230230714
4,US,20230230802,20230720,"{'country': 'US', 'doc-number': 18178664, 'dat...",Ultra High Purity Conditions for Atomic Scale ...,"{'orgname': 'Kurt J. Lesker Company', 'city': ...","[{'last-name': 'Rayner, JR.', 'first-name': 'G...",\nAn apparatus for atomic scale processing is ...,a reactor having inner and outer surfaces;\nwh...,"[apparatus, internal volume reactor, substrate...","[base pressure reactor, backfille process, pur...",20230230802
...,...,...,...,...,...,...,...,...,...,...,...,...
880,US,20230226141,20230720,"{'country': 'US', 'doc-number': 18157933, 'dat...",ANGIOTENSIN II ALONE OR IN COMBINATION FOR THE...,{'orgname': 'The George Washington University ...,"[{'last-name': 'CHAWLA', 'first-name': 'Lakhmi...","\nThe present invention relates, inter alia, t...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"[method, invention, effective, treatment, outp...",[],20230226141
881,US,20230226142,20230720,"{'country': 'US', 'doc-number': 18166956, 'dat...",IONIC SELF-ASSEMBLING PEPTIDES,"{'orgname': '3-D MATRIX, LTD.', 'city': 'Tokyo...","[{'last-name': 'Gil', 'first-name': 'Eun Seok'...",\nProvided herein are ionic self-assembling pe...,\n\na) an N-terminal functional group selected...,"[ionic self-assembling peptide, method, pharma...","[administer solution, pharmaceutical compositi...",20230226142
882,US,20230226143,20230720,"{'country': 'US', 'doc-number': 18011718, 'dat...",PHARMACEUTICAL COMPOSITION COMPRISING A COMBIN...,"{'orgname': 'OCVIRK, Soren', 'city': 'Kranzber...","[{'last-name': 'Ocvirk', 'first-name': 'Sören'...",\nA pharmaceutical composition includes a comb...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n,"[pharmaceutical composition, gastrointestinal ...",[],20230226143
883,US,20230226144,20230720,"{'country': 'US', 'doc-number': 18186639, 'dat...",TOPICAL ANTIBIOTIC,"{'orgname': 'Concept Matrix Solutions', 'city'...","[{'last-name': 'LaRosa', 'first-name': 'Tony',...",\nProvided herein is a topical antibiotic comp...,"(a) at least one of bacitracin, bacitracin zin...","[skin surface, antibiotic agent, method, compo...","[sodium hydroxide, cetearyl alcohol, water, et...",20230226144


In [9]:
selected_columns3 = ['key','doc-number', 'country',  'application-reference', 'title', 'assignee', 'inventors']
selected_columns4 = ['key', 'abstract_entities']

selected_columns3 = [col.strip() for col in selected_columns3]
selected_columns4 = [col.strip() for col in selected_columns4]

df_patents = df_patent[selected_columns3]
df_patents_main = df_patents[['key','doc-number', 'country', 'title']]
df_patentsent= df_patent[selected_columns4]

df_patents_main.to_csv('./preprocessed_files/neo4j_patents_main.csv', index=False)



In [10]:

df_source_patent = df_patentsent

# Convert the string representation to a list
#(need to update)
#df_source_patent['abstract_entities'] = df_source_patent['abstract_entities'].apply(ast.literal_eval)

# Explode the 'abstract_entities' column
#(need to update)
df_exploded_patents = df_source_patent.explode('abstract_entities')

# Display the resulting DataFrame
print(df_exploded_patents)

#df_exploded_patents.to_csv('./preprocessed_files/neo4j_patents_abstract.csv', index=False)

             key             abstract_entities
0    20230227158  splice electrical power line
0    20230227158               power line tool
0    20230227158         electrical power line
0    20230227158      attachment point support
0    20230227158                        system
..           ...                           ...
883  20230226144                   composition
884  20230226145                        method
884  20230226145            autoimmune disease
884  20230226145                    disclosure
884  20230226145    pharmaceutical composition

[6612 rows x 2 columns]


In [11]:
#1st Flattening
def safe_eval(value):
    try:
        return literal_eval(value)
    except (ValueError, SyntaxError):
        return value

def flatten_nested_columns(df, column_name):
    # Use safe_eval to handle malformed data
    df[column_name] = df[column_name].apply(safe_eval)
    
    # Flatten the nested dictionary
    df_flat = pd.concat([df.drop(column_name, axis=1),
                        json_normalize(df[column_name])], axis=1)
    
    return df_flat

# Your data

# Create a DataFrame
df = df_patents[['key','application-reference']]

# Flatten the specified nested column
df_flat = flatten_nested_columns(df, 'application-reference')

# Display the flattened DataFrame
print(df_flat)

df_flat.to_csv('./preprocessed_files/neo4j_patents_application_reference.csv', index=False)


             key country  doc-number      date
0    20230227158      US    18123405  20230320
1    20230229051      US    17577538  20220118
2    20230229087      US    18098167  20230118
3    20230230714      US    18010358  20220817
4    20230230802      US    18178664  20230306
..           ...     ...         ...       ...
880  20230226141      US    18157933  20230123
881  20230226142      US    18166956  20230209
882  20230226143      US    18011718  20200630
883  20230226144      US    18186639  20230320
884  20230226145      US    17379987  20210719

[885 rows x 4 columns]


/var/folders/mv/580ckk8j41q6_lsqh0dwhmfw0000gn/T/ipykernel_84767/1706582991.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].apply(safe_eval)


In [12]:
#2nd Flattening
# data
df = df_patents[['key','assignee']]

# Flatten the specified nested column
df_flat = flatten_nested_columns(df, 'assignee')

# Display the flattened DataFrame
print(df_flat)


df_flat.to_csv('./preprocessed_files/neo4j_patents_assignee.csv', index=False)

             key                                            orgname  \
0    20230227158                                    Beirobotics LLC   
1    20230229051                                      FURCIFER INC.   
2    20230229087                                      Nanosys, Inc.   
3    20230230714                     Ultra Safe Nuclear Corporation   
4    20230230802                             Kurt J. Lesker Company   
..           ...                                                ...   
880  20230226141  The George Washington University a Congression...   
881  20230226142                                   3-D MATRIX, LTD.   
882  20230226143                                      OCVIRK, Soren   
883  20230226144                           Concept Matrix Solutions   
884  20230226145                                     BIOINCEPT, LLC   

                city state country  
0           Richmond    VA      US  
1            FREMONT    CA      US  
2           Milpitas    CA      US  

/var/folders/mv/580ckk8j41q6_lsqh0dwhmfw0000gn/T/ipykernel_84767/1706582991.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].apply(safe_eval)


In [13]:
#3rd Flattening

def flatten_nested_nested_columns(df, column_name):
    df[column_name] = df[column_name].apply(safe_eval)
    
    # Create an empty DataFrame to store the flattened results
    df_flat = pd.DataFrame()
    
    # Iterate over each row
    for _, row in df.iterrows():
        # Flatten the nested column using json_normalize for each row
        df_row_flat = pd.json_normalize(row[column_name]).reset_index(drop=True)
        
        # Add the id_doc-number to the flattened DataFrame
        df_row_flat['key'] = row['key']
        
        # Concatenate the results for each row
        df_flat = pd.concat([df_flat, df_row_flat], ignore_index=True)
    
    return df_flat

# Extract specific columns
df = df_patents[['key', 'inventors']]

# Flatten the specified nested column
df_flat = flatten_nested_nested_columns(df, 'inventors')

# Display the flattened DataFrame
print(df_flat)


df_flat.to_csv('./preprocessed_files/neo4j_patents_inventors.csv', index=False)


/var/folders/mv/580ckk8j41q6_lsqh0dwhmfw0000gn/T/ipykernel_84767/3696540425.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].apply(safe_eval)


        last-name       first-name                city state country  \
0           Beiro  Michael Kenneth            Richmond    VA      US   
1     Corbin, III      Alvin Leroy             Dillwyn    VA      US   
2           Coble   Chase Hamilton  North Chesterfield    VA      US   
3           Schul     David Carson            Richmond    VA      US   
4            WANG             JIAN             FREMONT    CA      US   
...           ...              ...                 ...   ...     ...   
2750       Ocvirk            Sören           Kranzberg   NaN      DE   
2751       LaRosa             Tony      Woodland Hills    CA      US   
2752     Davidson           Robert      Woodland Hills    CA      US   
2753         Reid            David     Woodland Hillls    CA      US   
2754       Barnea         Eytan R.            New York    NY      US   

              key  
0     20230227158  
1     20230227158  
2     20230227158  
3     20230227158  
4     20230229051  
...           .

In [14]:
#similarity vector

import pandas as pd
import numpy as np
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity

# Load the Universal Sentence Encoder model
use_model_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
use_embed = hub.load(use_model_url)

# Sample dataframes (replace these with your actual dataframes)
df_exploded_sbir=df_exploded_sbir.head(800)
#sample patent data for simalrity purpose
df_exploded_patents=df_exploded_patents.head(800)

# Function to calculate vectors for phrases
def calculate_vectors(dataframe):
    vectors = []
    for phrase in dataframe['abstract_entities']:
        if pd.notna(phrase):  # Check for NaN values in the array
            vector = use_embed([phrase[0]])[0].numpy()  # Assuming you want to use the first element of the array
            vectors.append(vector)
        else:
            vectors.append(np.nan)
    return vectors


# Calculate vectors for phrases in each dataframe
vectors_sbir = calculate_vectors(df_exploded_sbir)
vectors_patents = calculate_vectors(df_exploded_patents)

# Store vectors as lists in the dataframes
df_exploded_sbir['vectors_sbir'] = vectors_sbir
df_exploded_patents['vectors_patents'] = vectors_patents

df_exploded_sbir.to_csv('./preprocessed_files/neo4j_sbir_abstract.csv', index=False)

df_exploded_patents.to_csv('./preprocessed_files/neo4j_patents_abstract.csv', index=False)

# Display the result dataframes
print(df_exploded_sbir.head(5), df_exploded_patents.head(5))

/var/folders/mv/580ckk8j41q6_lsqh0dwhmfw0000gn/T/ipykernel_84767/3659769068.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_exploded_sbir['vectors_sbir'] = vectors_sbir


   key                                 abstract_entities  \
0    0                                         condition   
1    0                                    signal density   
2    0                                     signal mingle   
3    0             signal intermodulation product desire   
4    0  Bandwidth ( ibw ) receiver wide bit-depth reduce   

                                        vectors_sbir  
0  [0.0214958, -0.06184944, 0.046937063, 0.047726...  
1  [0.0317812, -0.022999573, 0.060669057, 0.03845...  
2  [0.0317812, -0.022999573, 0.060669057, 0.03845...  
3  [0.0317812, -0.022999573, 0.060669057, 0.03845...  
4  [0.00612273, -0.07688116, 0.04913619, 0.060996...              key             abstract_entities  \
0  20230227158  splice electrical power line   
0  20230227158               power line tool   
0  20230227158         electrical power line   
0  20230227158      attachment point support   
0  20230227158                        system   

                    

In [15]:
# delete any csv from neo4j director if required
!rm '/Users/prakhar/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-c1b4f9c3-baa6-45d6-a13c-31ec4ba4c393/import'/*.csv
#!ls '/Users/prakhar/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-c1b4f9c3-baa6-45d6-a13c-31ec4ba4c393/import'

In [16]:
### Neo4j Data prep for PS
# rm '/Users/prakhar/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-c1b4f9c3-baa6-45d6-a13c-31ec4ba4c393/import'/*.csv

!cp './preprocessed_files/neo4j_sbir_main.csv' '/Users/prakhar/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-c1b4f9c3-baa6-45d6-a13c-31ec4ba4c393/import' && \
cp './preprocessed_files/neo4j_sbir_abstract.csv' '/Users/prakhar/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-c1b4f9c3-baa6-45d6-a13c-31ec4ba4c393/import' && \
cp './preprocessed_files/neo4j_patents_main.csv' '/Users/prakhar/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-c1b4f9c3-baa6-45d6-a13c-31ec4ba4c393/import' && \
cp './preprocessed_files/neo4j_patents_abstract.csv' '/Users/prakhar/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-c1b4f9c3-baa6-45d6-a13c-31ec4ba4c393/import' && \
cp './preprocessed_files/neo4j_patents_application_reference.csv' '/Users/prakhar/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-c1b4f9c3-baa6-45d6-a13c-31ec4ba4c393/import' && \
cp './preprocessed_files/neo4j_patents_assignee.csv' '/Users/prakhar/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-c1b4f9c3-baa6-45d6-a13c-31ec4ba4c393/import' && \
cp './preprocessed_files/neo4j_patents_inventors.csv' '/Users/prakhar/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-c1b4f9c3-baa6-45d6-a13c-31ec4ba4c393/import' && \
cp './preprocessed_files/neo4j_similarity_scores.csv' '/Users/prakhar/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-c1b4f9c3-baa6-45d6-a13c-31ec4ba4c393/import' && \
ls "/Users/prakhar/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-c1b4f9c3-baa6-45d6-a13c-31ec4ba4c393/import"

2                                       neo4j_patents_main.csv
neo4j_patents_abstract.csv              neo4j_sbir_abstract.csv
neo4j_patents_application_reference.csv neo4j_sbir_main.csv
neo4j_patents_assignee.csv              neo4j_similarity_scores.csv
neo4j_patents_inventors.csv


In [17]:
# Define the Cypher query to load data and nodes from SBIR
cypher_query_sbir1 = """
LOAD CSV WITH HEADERS FROM 'file:///neo4j_sbir_main.csv' AS row

// Create Location nodes
MERGE (location:Location {
  key: COALESCE(toInteger(row.key), 0),
  city: COALESCE(row.City, 'Unknown'),
  state: COALESCE(row.State, 'Unknown'),
  zip: COALESCE(row.Zip, 'Unknown'),
  address1: COALESCE(row.Address1, 'Unknown'),
  address2: COALESCE(row.Address2, 'Unknown')
})

// Create Agency nodes
MERGE (agency:Agency {
  key: COALESCE(toInteger(row.key), 0),
  name: COALESCE(row.Agency, 'Unknown'),
  branch: COALESCE(row.Branch, 'Unknown'),
  website: COALESCE(row.`Company Website`, 'Unknown')
})

// Create Abstract Topic nodes
MERGE (award:AWARD {
  key: COALESCE(toInteger(row.key), 0),
  topic_code: COALESCE(row.`Topic Code`, 'Unknown'),
  award_title: COALESCE(row.`Award Title`, 'Unknown'),
  award_year: COALESCE(row.`Award Year`, 'Unknown'),
  award_amount: COALESCE(row.`Award Amount`,'Unknown')
})

// Create Principal nodes
MERGE (principal:Principal {
  key: COALESCE(toInteger(row.key), 0),
  company: COALESCE(row.Company, 'Unknown'),
  pi_name: COALESCE(row.`PI Name`, 'Unknown'),
  pi_title: COALESCE(row.`PI Title`, 'Unknown'),
  pi_phone: COALESCE(row.`PI Phone`, 'Unknown'),
  pi_email: COALESCE(row.`PI Email`, 'Unknown'),
  hubzone_owned: COALESCE(row.`HUBZone Owned`, 'Unknown'),
  socially_economically_disadvantaged: COALESCE(row.`Socially and Economically Disadvantaged`, 'Unknown'),
  women_owned: COALESCE(row.`Women Owned`, 'Unknown')
})

// Create relationships
CREATE (agency)-[:OVERSEES]->(award)
CREATE (award)-[:BELONGSTO]->(principal)
CREATE (principal)-[:WORKSIN]->(location)
"""
#granted to relationship rather belongs to

# Execute the Cypher query
graph.run(cypher_query_sbir1)

# Define the Cypher query to load data from the second CSV file and create nodes
cypher_query_sbir2 = """
LOAD CSV WITH HEADERS FROM 'file:///neo4j_sbir_abstract.csv' AS row

// Create Abstract_Entities nodes
MERGE (abstract_entities:Abstract_Entities {
  key: COALESCE(toInteger(row.key), 0),
  abstract_entities: COALESCE(row.`abstract_entities`, 'Unknown'),
  vectors_sbir: COALESCE(row.vectors_sbir, 'Unknown')
})
"""

# Execute the Cypher query
graph.run(cypher_query_sbir2)

# Create relationships between nodes from the second file and existing nodes from the first file
cypher_query_sbir3 = """
MATCH (award:AWARD), (principal:Principal), (abstract_entities:Abstract_Entities)
WHERE award.key = principal.key AND principal.key = abstract_entities.key
CREATE (award)-[:CONTAINS]->(abstract_entities)
CREATE (principal)-[:RESEARCHED]->(abstract_entities)
"""

# Execute the Cypher query
graph.run(cypher_query_sbir3)

(No data)

In [18]:
# Define the Cypher query to load data and nodes from Patents
cypher_query_patents1 = """
LOAD CSV WITH HEADERS FROM 'file:///neo4j_patents_main.csv' AS row

// Create Title nodes
MERGE (title_pt:Title_PT {
  key: COALESCE(toInteger(row.key), 0),
  doc_number: COALESCE(row.`doc-number`, 'Unknown'),
  title: COALESCE(row.title, 'Unknown'),
  country: COALESCE(row.country, 'Unknown')
})

"""
# Execute the Cypher query
graph.run(cypher_query_patents1)


# Define the Cypher query to load data from the second CSV file and create nodes
cypher_query_patents2 = """
LOAD CSV WITH HEADERS FROM 'file:///neo4j_patents_application_reference.csv' AS row

// Create appplication ref nodes
MERGE (application_ref_pt:Application_Ref_PT {
  key: COALESCE(toInteger(row.key), 0),
  doc_number: COALESCE(row.`doc-number`, 'Unknown'),
  country: COALESCE(row.country, 'Unknown'),
  date: COALESCE(row.date, 'Unknown')
})
"""

# Execute the Cypher query
graph.run(cypher_query_patents2)


# Define the Cypher query to load data from the second CSV file and create nodes
cypher_query_patents3 = """
LOAD CSV WITH HEADERS FROM 'file:///neo4j_patents_assignee.csv' AS row

// Create assignee nodes
MERGE (assignee_pt:Assignee_PT {
  key: COALESCE(toInteger(row.key), 0),
  orgname: COALESCE(row.orgname, 'Unknown'),
  city: COALESCE(row.city, 'Unknown'),
  state: COALESCE(row.state, 'Unknown'),
  country: COALESCE(row.country, 'Unknown')
});
"""

# Execute the Cypher query
graph.run(cypher_query_patents3)

# Define the Cypher query to load data from the second CSV file and create nodes
cypher_query_patents4 = """
LOAD CSV WITH HEADERS FROM 'file:///neo4j_patents_inventors.csv' AS row

// Create assignee nodes
MERGE (inventors_pt:Inventors_PT {
  key: COALESCE(toInteger(row.key), 0),
  first_name: COALESCE(row.`first-name`, 'Unknown'),
  last_name: COALESCE(row.`last-name`, 'Unknown'),
  city: COALESCE(row.city, 'Unknown'),
  state: COALESCE(row.state, 'Unknown'),
  country: COALESCE(row.country, 'Unknown') 
})

"""

# Execute the Cypher query
graph.run(cypher_query_patents4)


# Define the Cypher query to load data from the second CSV file and create nodes
cypher_query_patents5 = """
LOAD CSV WITH HEADERS FROM 'file:///neo4j_patents_abstract.csv' AS row

// Create abstract_entities nodes
MERGE (abstract_entities_pt:Abstract_Entities_PT {
  key: COALESCE(toInteger(row.key), 0),
  vectors_patents: COALESCE(row.vectors_patents, 'Unknown'),
  abstract_entities: COALESCE(row.abstract_entities, 'Unknown')
})
"""

# Execute the Cypher query
graph.run(cypher_query_patents5)

# Create relationships between nodes from the second file and existing nodes from the first file
cypher_query_patents6 = """
MATCH (title_pt:Title_PT) , (assignee_pt:Assignee_PT), (inventors_pt:Inventors_PT), (applications_ref_pt:Application_Ref_PT), (abstract_entities_pt:Abstract_Entities_PT)
WHERE title_pt.key = assignee_pt.key AND  assignee_pt.key = inventors_pt.key AND inventors_pt.key = applications_ref_pt.key AND applications_ref_pt.key = abstract_entities_pt.key
CREATE (title_pt)-[:ASSIGN]->(assignee_pt)
CREATE (title_pt)-[:INVENTOR]->(inventors_pt)
CREATE (inventors_pt)-[:RESEARCHED]->(abstract_entities_pt)
CREATE (applications_ref_pt)-[:CONTAINS]->(abstract_entities_pt)
"""

# Execute the Cypher query
graph.run(cypher_query_patents6)

(No data)

In [19]:
# Create relationships between nodes from the second file and existing nodes from the first file
cypher_query_patents8 = """
MATCH (title_pt:Title_PT) , (assignee_pt:Assignee_PT), (inventors_pt:Inventors_PT), (applications_ref_pt:Application_Ref_PT), (abstract_entities_pt:Abstract_Entities_PT)
WHERE title_pt.key = assignee_pt.key AND  assignee_pt.key = inventors_pt.key AND inventors_pt.key = applications_ref_pt.key AND applications_ref_pt.key = abstract_entities_pt.key
CREATE (title_pt)-[:ASSIGN]->(assignee_pt)
CREATE (title_pt)-[:INVENTOR]->(inventors_pt)
CREATE (inventors_pt)-[:RESEARCHED]->(abstract_entities_pt)
CREATE (applications_ref_pt)-[:CONTAINS]->(abstract_entities_pt)
"""

# Execute the Cypher query
graph.run(cypher_query_patents8)

(No data)

In [20]:
#Cypher queries to use Similarity Vector in neo4j
cypher_query_similarityscore_test = """
MATCH (a:Abstract_Entities)
MATCH (b:Abstract_Entities_PT)
WHERE a <> b
WITH a, b,
     [x IN split(substring(a.vectors_sbir, 1, size(a.vectors_sbir)-1), ' ') | toFloat(x)] AS vectorA,
     [y IN split(substring(b.vectors_patents, 1, size(b.vectors_patents)-1), ' ') | toFloat(y)] AS vectorB
RETURN a.key,a.abstract_entities ,b.key, b.abstract_entities, gds.similarity.jaccard(vectorA, vectorB) AS jaccardSimilarity
ORDER BY jaccardSimilarity DESC
LIMIT 10;
"""

# Execute the Cypher query
graph.run(cypher_query_similarityscore_test)

a.key,a.abstract_entities,b.key,b.abstract_entities,jaccardSimilarity
0,condition,20230225238,control signal,1.0
0,condition,20230225239,classification,1.0
0,condition,20230225236,cultivation process,1.0


In [ ]:
#Cypher queries to use Similarity Vector in neo4j
cypher_query_similarityscore_test = """
MATCH (a:Abstract_Entities)
MATCH (b:Abstract_Entities_PT)
WHERE a <> b
WITH a, b,
     [x IN split(substring(a.vectors_sbir, 1, size(a.vectors_sbir)-1), ' ') | toFloat(x)] AS vectorA,
     [y IN split(substring(b.vectors_patents, 1, size(b.vectors_patents)-1), ' ') | toFloat(y)] AS vectorB
RETURN a.key,a.abstract_entities ,b.key, b.abstract_entities, gds.similarity.jaccard(vectorA, vectorB) AS jaccardSimilarity
ORDER BY jaccardSimilarity DESC
LIMIT 10;
"""

# Execute the Cypher query
graph.run(cypher_query_similarityscore_test)